# <a id='toc1_'></a>[Post-processing of results (second part)](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Post-processing of results (second part)](#toc1_)    
  - [Preliminaries](#toc1_1_)    
    - [Import libraries](#toc1_1_1_)    
    - [Utility functions](#toc1_1_2_)    
    - [Utilities for the summarized version](#toc1_1_3_)    
    - [Utilities for the full version](#toc1_1_4_)    
    - [Load the results](#toc1_1_5_)    
  - [Basin: E12GM](#toc1_2_)    
    - [Versione full](#toc1_2_1_)    
      - [PCMCI](#toc1_2_1_1_)    
      - [TEFS](#toc1_2_1_2_)    
    - [Versione summarized](#toc1_2_2_)    
      - [PCMCI](#toc1_2_2_1_)    
      - [TEFS](#toc1_2_2_2_)    
    - [Versione full senza CMI](#toc1_2_3_)    
      - [TEFS](#toc1_2_3_1_)    
    - [TEFS as wrapper on E12GM](#toc1_2_4_)    
    - [Linking the wrapper to the original filter method](#toc1_2_5_)    
  - [Basin: Ticino](#toc1_3_)    
    - [Versione full](#toc1_3_1_)    
      - [PCMCI](#toc1_3_1_1_)    
      - [TEFS](#toc1_3_1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Preliminaries](#toc0_)

### <a id='toc1_1_1_'></a>[Import libraries](#toc0_)

In [1]:
import os

os.chdir(os.path.dirname(os.path.abspath(__file__)))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import thesis.constants as constants
import thesis.file_management as file_management
from sklearn.model_selection import (
    KFold,
    TimeSeriesSplit,
)
from tefs.metrics import regression_analysis
from thesis import datasets_and_configurations_loaders

Set the retina resolution

In [2]:
%config InlineBackend.figure_format = 'retina'

Enable the use of LaTeX for plots.

In [3]:
plt.rc("text", usetex=True)

### <a id='toc1_1_2_'></a>[Utility functions](#toc0_)

General purpose formatter functions, valid for all stylers.

In [4]:
def makecell_code_formatter(x):
    """
    Format a string to be used in a LaTeX table cell. Specifically, given a list of names, it will format them as a
    single column with each name in a separate row and in a monospaced font.
    """
    elements = x.split(" ")
    formatted_elements = [f"\\texttt{{{element}}}" for element in elements]
    return "\\makecell[l]{" + "\\\\ ".join(formatted_elements) + "}"


def format_time(seconds):
    """
    Format a time in seconds to a human-readable format.
    """
    return f"{seconds:.3f}s"


def highlight_row(s, row_indexes, color):
    """
    Highlight the given row indexes of Series s with the given color.
    """
    if "test" in s.name.lower():
        return ["" for _ in s]  # No styling for columns with "test" in their name
    return ["background-color: " + color if i in row_indexes else "" for i in range(len(s))]


def color_direction(v):
    """
    Color the text of a cell according to the direction of the value.
    """
    color = "black"
    if v == "backward":
        color = "red"
    elif v == "forward":
        color = "blue"
    return f"color: {color}"

### <a id='toc1_1_3_'></a>[Utilities for the summarized version](#toc0_)

Some utilities are specific to the summarized version of the table of results.

For the PCMCI version.

In [5]:
def make_pcmci_pretty(styler):
    styler.format(subset=["score_r2_lag", "score_r2_lag_ar"], precision=3)
    styler.background_gradient(cmap="Greens", subset=["score_r2_lag", "score_r2_lag_ar"], vmin=0, vmax=0.5)
    styler.format(subset=["execution_time"], precision=2)
    # styler.format(formatter=format_time, subset=["execution_time"])
    styler.background_gradient(cmap="Reds", subset=["execution_time"], vmax=8000)
    return styler


from pandas.io.formats.style_render import _escape_latex

# I lost too much time trying to figure out why I can't format the "names" of the indexes (to escape them)
# The guy who wrote it didn't allow for this possibility, but thankfully (https://stackoverflow.com/questions/72716879/is-there-a-function-to-format-the-index-name-in-a-pandas-styler-dataframe-style)
# he proposed a workaround and opened a github issue (https://github.com/pandas-dev/pandas/issues/47489)
# But he didn't realize that this doesn't work on multi-indexes, so I had to modify his code a bit


def export_pcmci_df_to_latex(df, target_file, code_escaped_columns=[]):
    temp_df = df.copy()
    for level in range(temp_df.index.nlevels):
        if temp_df.index.get_level_values(level).name is not None:
            temp_df.index.set_names(_escape_latex(temp_df.index.get_level_values(level).name), level=level, inplace=True)
    for level in range(temp_df.columns.nlevels):
        if temp_df.columns.get_level_values(level).name is not None:
            temp_df.columns.set_names(_escape_latex(temp_df.columns.get_level_values(level).name), level=level, inplace=True)

    with open(target_file, "w") as f:
        f.write(
            temp_df.style.pipe(make_pcmci_pretty)
            .format_index(escape="latex", axis="index")
            .format_index(escape="latex", axis="columns")
            .format(formatter=makecell_code_formatter, escape="latex", subset=code_escaped_columns)
            .to_latex(hrules=True, clines="all;index", convert_css=True, column_format="cclccccrr")
        )

For the TEFS version.

In [6]:
def make_te_pretty(styler):
    styler.format(subset=["score_r2_lag", "score_r2_lag_ar"], precision=3)
    styler.background_gradient(cmap="Greens", subset=["score_r2_lag", "score_r2_lag_ar"], vmin=0, vmax=0.5)
    styler.format(subset=["execution_time"], precision=2)
    styler.background_gradient(cmap="Reds", subset=["execution_time"], vmax=8000)
    return styler


def export_te_df_to_latex(df, target_file, code_escaped_columns=[]):
    temp_df = df.copy()
    for level in range(temp_df.index.nlevels):
        if temp_df.index.get_level_values(level).name is not None:
            temp_df.index.set_names(_escape_latex(temp_df.index.get_level_values(level).name), level=level, inplace=True)
    for level in range(temp_df.columns.nlevels):
        if temp_df.columns.get_level_values(level).name is not None:
            temp_df.columns.set_names(_escape_latex(temp_df.columns.get_level_values(level).name), level=level, inplace=True)

    with open(target_file, "w") as f:
        f.write(
            temp_df.style.pipe(make_te_pretty)
            .format_index(escape="latex", axis="index")
            .format_index(escape="latex", axis="columns")
            .format(formatter=makecell_code_formatter, escape="latex", subset=code_escaped_columns)
            .to_latex(hrules=True, clines="all;index", convert_css=True, column_format="cclccccrr")
        )

### <a id='toc1_1_4_'></a>[Utilities for the full version](#toc0_)

Some utilities are specific to the full version of the table of results.

For the PCMCI version.

In [7]:
def make_pcmci_all_pretty(styler):
    styler.format(subset=["score_r2", "score_r2_lag", "score_r2_lag_ar"], precision=3)
    styler.background_gradient(cmap="Greens", subset=["score_r2", "score_r2_lag", "score_r2_lag_ar"], vmin=0, vmax=0.5)
    styler.format(formatter=format_time, subset=["execution_time"])
    styler.background_gradient(cmap="Reds", subset=["execution_time"], vmax=8000)
    return styler


def export_results_dataframe_pcmci(df: pd.DataFrame, target_file: str):
    with open(target_file, "w") as f:
        f.write(
            df.style.pipe(make_pcmci_all_pretty)
            .format_index(escape="latex", axis=1)
            .format(formatter=makecell_code_formatter, escape="latex", subset=["selected_features", "dataset", "algorithm", "independencetest"])
            .to_latex(hrules=True, clines="all;data", convert_css=True, column_format="llccclllcr")
        )

For the TEFS version.

In [8]:
# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.html
# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.format.html
# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
# https://www.youtube.com/watch?v=JGefS6WPm1E
# https://tex.stackexchange.com/questions/2441/how-to-add-a-forced-line-break-inside-a-table-cell
def make_te_all_pretty(styler):
    styler.format(subset=["score_r2", "score_r2_lag", "score_r2_lag_ar"], precision=3)
    styler.background_gradient(cmap="Greens", subset=["score_r2", "score_r2_lag", "score_r2_lag_ar"], vmin=0, vmax=0.5)
    #styler.map(color_direction, subset=["direction"])
    styler.format(formatter=format_time, subset=["execution_time"])
    styler.background_gradient(cmap="Reds", subset=["execution_time"], vmax=8000)
    # styler.apply(highlight_row, row_indexes=[5,7], color='yellow', axis=0)
    return styler


def export_results_dataframe_te(df: pd.DataFrame, target_file: str):
    with open(target_file, "w") as f:
        f.write(
            df.style.pipe(make_te_all_pretty)
            .format_index(escape="latex", axis=1)
            .format(formatter=makecell_code_formatter, escape="latex", subset=["selected_features", "dataset", "direction"])
            .to_latex(hrules=True, clines="all;data", convert_css=True, column_format="llccclcclr")
        )

### <a id='toc1_1_5_'></a>[Load the results](#toc0_)

Load the previously exported pandas dataframes containing the results of the analysis.

In [9]:
results_e12gm_pcmci = file_management.load_from_pkl_file(os.path.join(constants.path_table_objects, "results_table_e12gm_pcmci.pkl"))
results_e12gm_te = file_management.load_from_pkl_file(os.path.join(constants.path_table_objects, "results_table_e12gm_te.pkl"))
results_ticino_pcmci = file_management.load_from_pkl_file(os.path.join(constants.path_table_objects, "results_table_ticino_pcmci.pkl"))
results_ticino_te = file_management.load_from_pkl_file(os.path.join(constants.path_table_objects, "results_table_ticino_te.pkl"))

results_e12gm_noCMI_te = file_management.load_from_pkl_file(os.path.join(constants.path_table_objects, "results_table_e12gm_noCMI_te.pkl"))

In [10]:
from scripts.run_benchmark import baseline

baseline

,Emiliani1,Emiliani2,GardaMincio,Ticino
solo (lag=0),0.286071,0.243534,0.171307,0.154807
"solo (lag=0,1)",0.284043,0.306111,0.191933,0.177646
solo + extended (lag=0),0.344448,0.292431,0.185493,0.162634
"solo + extended (lag=0,1)",0.373261,0.292733,0.169552,0.167326
ar(1),0.275232,0.292035,0.289653,0.199807
"ar(1) + solo (lag=0,1)",0.424901,0.432553,0.437795,0.331817
"ar(1) + solo + extended (lag=0,1)",0.468767,0.430417,0.413616,0.330220


## <a id='toc1_3_'></a>[Basin: Ticino](#toc0_)

### <a id='toc1_3_1_'></a>[Full version](#toc0_)

#### <a id='toc1_3_1_1_'></a>[PCMCI](#toc0_)

In [11]:
target_file = os.path.join(constants.path_table_tex, "ticino_pcmci_full.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_results_dataframe_pcmci(results_ticino_pcmci, target_file)
results_ticino_pcmci.style.pipe(make_pcmci_all_pretty)

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,algorithm,independencetest,lag,execution_time
0,cyclostationary_mean_tg_0 cyclostationary_mean_rr_4w_0,0.149,0.149,0.149,normal,pcmci_plus,cmiknn,0,32.662s
1,cyclostationary_mean_tg_0 cyclostationary_mean_rr_4w_0,0.149,0.149,0.149,normal,pcmci_plus,parcorr,0,0.018s
2,cyclostationary_mean_tg_0,0.138,0.188,0.331,normal,pcmci_plus,cmiknn,1,112.996s
3,cyclostationary_mean_tg_0,0.138,0.188,0.331,normal,pcmci_plus,parcorr,1,0.069s
4,cyclostationary_mean_HS_0 cyclostationary_mean_tg_0 cyclostationary_mean_rr_4w_0,0.149,0.149,0.149,snowlakes,pcmci_plus,cmiknn,0,264.168s
5,cyclostationary_mean_tg_0 cyclostationary_mean_rr_4w_0,0.149,0.149,0.149,snowlakes,pcmci_plus,parcorr,0,0.146s
6,cyclostationary_mean_tg_0,0.138,0.188,0.331,snowlakes,pcmci_plus,cmiknn,1,347.146s
7,cyclostationary_mean_tg_0,0.138,0.188,0.331,snowlakes,pcmci_plus,parcorr,1,0.285s


#### <a id='toc1_3_1_2_'></a>[TEFS](#toc0_)

In [12]:
target_file = os.path.join(constants.path_table_tex, "ticino_te_full.tex")
export_results_dataframe_te(results_ticino_te, target_file)
results_ticino_te.style.pipe(make_te_all_pretty)

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,lagfeatures,lagtarget,direction,execution_time
0,cyclostationary_mean_tg_0,0.138,0.138,0.346,normal,[0],[1],backward,0.401s
1,cyclostationary_mean_tg_0,0.138,0.138,0.346,normal,[0],[1],forward,0.372s
2,cyclostationary_mean_tg_0,0.138,0.188,0.331,normal,"[0, 1]",[1],backward,0.415s
3,cyclostationary_mean_tg_0,0.138,0.188,0.331,normal,"[0, 1]",[1],forward,0.388s
4,cyclostat_level_Lugano cyclostationary_mean_tg_0,0.194,0.194,0.360,snowlakes,[0],[1],backward,1.650s
5,cyclostat_level_Lugano cyclostationary_mean_tg_0,0.194,0.194,0.360,snowlakes,[0],[1],forward,1.612s
6,cyclostationary_mean_HS_0 cyclostationary_mean_tg_2,0.067,0.083,0.309,snowlakes,"[0, 1]",[1],backward,1.777s
7,cyclostat_level_Lugano cyclostationary_mean_tg_0,0.194,0.238,0.345,snowlakes,"[0, 1]",[1],forward,1.499s


## <a id='toc1_2_'></a>[Basin: E12GM](#toc0_)

### <a id='toc1_2_1_'></a>[Full version](#toc0_)

#### <a id='toc1_2_1_1_'></a>[PCMCI](#toc0_)

In [13]:
target_file = os.path.join(constants.path_table_tex, "e12gm_pcmci_full.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_results_dataframe_pcmci(results_e12gm_pcmci, target_file)
results_e12gm_pcmci.style.pipe(make_pcmci_all_pretty)

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,algorithm,independencetest,lag,execution_time
0,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_1w_16 E1cyclostationary_mean_rr_24w_2,0.286,0.286,0.286,df_E1,pcmci_plus,cmiknn,0,384.152s
1,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_24w_2,0.289,0.289,0.289,df_E1,pcmci_plus,parcorr,0,0.046s
2,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_1w_16,0.251,0.255,0.420,df_E1,pcmci_plus,cmiknn,1,1308.914s
3,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.257,0.265,0.421,df_E1,pcmci_plus,parcorr,1,0.274s
4,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_rr_1w_16 E1cyclostationary_mean_rr_24w_2,0.181,0.181,0.181,df_E1allfeatures,pcmci_plus,cmiknn,0,5532.618s
5,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_rr_24w_2 E2cyclostationary_mean_tg_0,0.254,0.254,0.254,df_E1allfeatures,pcmci_plus,parcorr,0,0.678s
6,E1cyclostationary_mean_rr_4w_1,0.151,0.148,0.368,df_E1allfeatures,pcmci_plus,cmiknn,1,7726.129s
7,E1cyclostationary_mean_rr_4w_1,0.151,0.148,0.368,df_E1allfeatures,pcmci_plus,parcorr,1,10.819s
8,E2cyclostationary_mean_tg_0 E2cyclostationary_mean_rr_8w_0 E2cyclostationary_mean_rr_4w_5,0.230,0.230,0.230,df_E2,pcmci_plus,cmiknn,0,810.721s
9,E2cyclostationary_mean_tg_0 E2cyclostationary_mean_rr_4w_5,0.222,0.222,0.222,df_E2,pcmci_plus,parcorr,0,0.058s


#### <a id='toc1_2_1_2_'></a>[TEFS](#toc0_)

In [14]:
target_file = os.path.join(constants.path_table_tex, "e12gm_te_full.tex")
export_results_dataframe_te(results_e12gm_te, target_file)
results_e12gm_te.style.pipe(make_te_all_pretty)

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,lagfeatures,lagtarget,direction,execution_time
0,E1cyclostationary_mean_rr_1w_16 E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.251,0.251,0.457,df_E1,[0],[1],backward,0.688s
1,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_1w_16,0.251,0.251,0.457,df_E1,[0],[1],forward,0.633s
2,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.257,0.265,0.421,df_E1,"[0, 1]",[1],backward,0.757s
3,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.257,0.265,0.421,df_E1,"[0, 1]",[1],forward,0.659s
4,GMcyclostationary_mean_tg_1w_0 E2cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_4w_1,0.231,0.231,0.405,df_E1allfeatures,[0],[1],backward,6.806s
5,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_1w_16,0.251,0.251,0.457,df_E1allfeatures,[0],[1],forward,6.430s
6,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.257,0.265,0.421,df_E1allfeatures,"[0, 1]",[1],backward,7.315s
7,E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_0,0.257,0.265,0.421,df_E1allfeatures,"[0, 1]",[1],forward,6.634s
8,E2cyclostationary_mean_rr_8w_0 E2cyclostationary_mean_tg_0,0.190,0.190,0.423,df_E2,[0],[1],backward,1.123s
9,E2cyclostationary_mean_tg_0 E2cyclostationary_mean_rr_8w_0,0.190,0.190,0.423,df_E2,[0],[1],forward,0.999s


### <a id='toc1_2_2_'></a>[Summarized version](#toc0_)

#### <a id='toc1_2_2_1_'></a>[PCMCI](#toc0_)

In [15]:
results_e12gm_pcmci

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,algorithm,independencetest,lag,execution_time
0,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.286071,0.286071,0.286071,df_E1,pcmci_plus,cmiknn,0,384.151514
1,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.289250,0.289250,0.289250,df_E1,pcmci_plus,parcorr,0,0.045593
2,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.251060,0.255466,0.419614,df_E1,pcmci_plus,cmiknn,1,1308.913636
3,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.256639,0.265190,0.420932,df_E1,pcmci_plus,parcorr,1,0.274470
4,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.181424,0.181424,0.181424,df_E1allfeatures,pcmci_plus,cmiknn,0,5532.617838
5,E1cyclostationary_mean_rr_4w_1 E1cyclostationa...,0.254281,0.254281,0.254281,df_E1allfeatures,pcmci_plus,parcorr,0,0.678409
6,E1cyclostationary_mean_rr_4w_1,0.150558,0.147686,0.367535,df_E1allfeatures,pcmci_plus,cmiknn,1,7726.128555
7,E1cyclostationary_mean_rr_4w_1,0.150558,0.147686,0.367535,df_E1allfeatures,pcmci_plus,parcorr,1,10.818769
8,E2cyclostationary_mean_tg_0 E2cyclostationary_...,0.230389,0.230389,0.230389,df_E2,pcmci_plus,cmiknn,0,810.720993
9,E2cyclostationary_mean_tg_0 E2cyclostationary_...,0.221522,0.221522,0.221522,df_E2,pcmci_plus,parcorr,0,0.057513


In [16]:
results_e12gm_pcmci["features_lag"] = results_e12gm_pcmci["lag"].map(
    {
        0: "contemporary",
        1: "contemporary + 1-lagged",
    }
)
results_e12gm_pcmci["features_set"] = results_e12gm_pcmci["dataset"].apply(lambda x: "all" if "all" in x else "single")

results_e12gm_pcmci["dataset"] = results_e12gm_pcmci["dataset"].apply(lambda x: x[3:5])

results_e12gm_pcmci = results_e12gm_pcmci\
    .drop(columns=["selected_features", "algorithm", "lag", "score_r2"])\
    .set_index(["dataset", "independencetest", "features_lag", "features_set"])\
    .unstack("features_set").sort_index(axis=1, ascending=False)  # fmt: off

results_e12gm_pcmci

score_r2_lag_ar            \
features_set                                              single       all   
dataset independencetest features_lag                                        
E1      cmiknn           contemporary                   0.286071  0.181424   
                         contemporary + 1-lagged        0.419614  0.367535   
        parcorr          contemporary                   0.289250  0.254281   
                         contemporary + 1-lagged        0.420932  0.367535   
E2      cmiknn           contemporary                   0.230389  0.297771   
                         contemporary + 1-lagged        0.415827  0.390651   
        parcorr          contemporary                   0.221522  0.191082   
                         contemporary + 1-lagged        0.401565  0.390651   
GM      cmiknn           contemporary                   0.176563  0.087925   
                         contemporary + 1-lagged        0.442869  0.406443   
        parcorr          contemporary                   0.176563  0.109771   
                         contemporary + 1-lagged        0.442869  0.406443   

                                                 score_r2_lag            \
features_set                                           single       all   
dataset independencetest features_lag                                     
E1      cmiknn           contemporary                0.286071  0.181424   
                         contemporary + 1-lagged     0.255466  0.147686   
        parcorr          contemporary                0.289250  0.254281   
                         contemporary + 1-lagged     0.265190  0.147686   
E2      cmiknn           contemporary                0.230389  0.297771   
                         contemporary + 1-lagged     0.239127  0.146193   
        parcorr          contemporary                0.221522  0.191082   
                         contemporary + 1-lagged     0.248450  0.146193   
GM      cmiknn           contemporary                0.176563  0.087925   
                         contemporary + 1-lagged     0.203489  0.119979   
        parcorr          contemporary                0.176563  0.109771   
                         contemporary + 1-lagged     0.203489  0.119979   

                                                 execution_time               
features_set                                             single          all  
dataset independencetest features_lag                                         
E1      cmiknn           contemporary                384.151514  5532.617838  
                         contemporary + 1-lagged    1308.913636  7726.128555  
        parcorr          contemporary                  0.045593     0.678409  
                         contemporary + 1-lagged       0.274470    10.818769  
E2      cmiknn           contemporary                810.720993  5081.931851  
                         contemporary + 1-lagged    1596.576730  5268.378825  
        parcorr          contemporary                  0.057513     0.341040  
                         contemporary + 1-lagged       0.314501     8.094075  
GM      cmiknn           contemporary                 87.160898  1826.917106  
                         contemporary + 1-lagged     504.895575  4688.384780  
        parcorr          contemporary                  0.016086     0.345339  
                         contemporary + 1-lagged       0.089332     9.835356

In [17]:
target_file = os.path.join(constants.path_table_tex, "e12gm_E1_pcmci.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_pcmci_df_to_latex(results_e12gm_pcmci.iloc[:4, :], target_file)
results_e12gm_pcmci.iloc[:4, :].style.pipe(make_pcmci_pretty)

In [18]:
target_file = os.path.join(constants.path_table_tex, "e12gm_E2_pcmci.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_pcmci_df_to_latex(results_e12gm_pcmci.iloc[4:8, :], target_file)
results_e12gm_pcmci.iloc[4:8, :].style.pipe(make_pcmci_pretty)

In [19]:
target_file = os.path.join(constants.path_table_tex, "e12gm_GM_pcmci.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_pcmci_df_to_latex(results_e12gm_pcmci.iloc[8:, :], target_file)
results_e12gm_pcmci.iloc[8:, :].style.pipe(make_pcmci_pretty)

#### <a id='toc1_2_2_2_'></a>[TEFS](#toc0_)

In [20]:
def check_list(cell):
    if cell == [0]:
        return "contemporary"
    elif cell == [0, 1]:
        return "contemporary + 1-lagged"

results_e12gm_te["features_lag"] = results_e12gm_te["lagfeatures"].apply(check_list)

In [21]:
results_e12gm_te["features_set"] = results_e12gm_te["dataset"].apply(lambda x: "all" if "all" in x else "single")
# df_e12gm_te["CMI"] = df_e12gm_te["dataset"].apply(lambda x: "noCMI" if "noCMI" in x else "yesCMI")
results_e12gm_te["dataset"] = results_e12gm_te["dataset"].apply(lambda x: x[3:5])

results_e12gm_te = results_e12gm_te\
    .drop(columns=["selected_features", "lagtarget", "lagfeatures", "score_r2"])\
    .set_index(["dataset", "direction", "features_lag", "features_set"])\
    .unstack("features_set").sort_index(axis=1, ascending=False)  # fmt: off

results_e12gm_te

score_r2_lag_ar            \
features_set                                       single       all   
dataset direction features_lag                                        
E1      backward  contemporary                   0.456840  0.405068   
                  contemporary + 1-lagged        0.420932  0.420932   
        forward   contemporary                   0.456840  0.456840   
                  contemporary + 1-lagged        0.420932  0.420932   
E2      backward  contemporary                   0.423374  0.423374   
                  contemporary + 1-lagged        0.390651  0.390651   
        forward   contemporary                   0.423374  0.423374   
                  contemporary + 1-lagged        0.390651  0.421809   
GM      backward  contemporary                   0.359131  0.457228   
                  contemporary + 1-lagged        0.410581  0.376746   
        forward   contemporary                   0.359131  0.391654   
                  contemporary + 1-lagged        0.410581  0.410581   

                                          score_r2_lag            \
features_set                                    single       all   
dataset direction features_lag                                     
E1      backward  contemporary                0.251060  0.231462   
                  contemporary + 1-lagged     0.265190  0.265190   
        forward   contemporary                0.251060  0.251060   
                  contemporary + 1-lagged     0.265190  0.265190   
E2      backward  contemporary                0.190334  0.190334   
                  contemporary + 1-lagged     0.146193  0.146193   
        forward   contemporary                0.190334  0.190334   
                  contemporary + 1-lagged     0.146193  0.146169   
GM      backward  contemporary                0.176563  0.157890   
                  contemporary + 1-lagged     0.050087  0.085362   
        forward   contemporary                0.176563  0.012959   
                  contemporary + 1-lagged     0.050087  0.050087   

                                          execution_time            
features_set                                      single       all  
dataset direction features_lag                                      
E1      backward  contemporary                  0.688195  6.806478  
                  contemporary + 1-lagged       0.757299  7.315034  
        forward   contemporary                  0.633147  6.430187  
                  contemporary + 1-lagged       0.659232  6.633933  
E2      backward  contemporary                  1.122526  7.054950  
                  contemporary + 1-lagged       1.219591  7.271728  
        forward   contemporary                  0.998714  5.996039  
                  contemporary + 1-lagged       1.053389  6.607501  
GM      backward  contemporary                  0.356708  6.942717  
                  contemporary + 1-lagged       0.399291  7.320499  
        forward   contemporary                  0.349754  6.280308  
                  contemporary + 1-lagged       0.356036  6.558476

In [22]:
target_file = os.path.join(constants.path_table_tex, "e12gm_E1_te.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_te_df_to_latex(results_e12gm_te.iloc[:4, :], target_file)
results_e12gm_te.iloc[:4, :].style.pipe(make_te_pretty)

In [23]:
target_file = os.path.join(constants.path_table_tex, "e12gm_E2_te.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_te_df_to_latex(results_e12gm_te.iloc[4:8, :], target_file)
results_e12gm_te.iloc[4:8, :].style.pipe(make_te_pretty)

In [24]:
target_file = os.path.join(constants.path_table_tex, "e12gm_GM_te.tex")
os.makedirs(os.path.dirname(target_file), exist_ok=True)
export_te_df_to_latex(results_e12gm_te.iloc[8:, :], target_file)
results_e12gm_te.iloc[8:, :].style.pipe(make_te_pretty)

### <a id='toc1_2_3_'></a>[Full version without CMI](#toc0_)

#### <a id='toc1_2_3_1_'></a>[TEFS](#toc0_)

In [25]:
target_file = os.path.join(constants.path_table_tex, "e12gm_noCMI_te_full.tex")
export_results_dataframe_te(results_e12gm_noCMI_te, target_file)
results_e12gm_noCMI_te.style.pipe(make_te_all_pretty)

,selected_features,score_r2,score_r2_lag,score_r2_lag_ar,dataset,lagfeatures,lagtarget,direction,execution_time
0,E1cyclostationary_mean_tg_12w_3 E1cyclostationary_mean_rr_1w_10 E1cyclostationary_mean_tg_8w_1 E1cyclostationary_mean_tg_6 E1cyclostationary_mean_tg_12w_2 E1cyclostationary_mean_rr_1w_9 E1cyclostationary_mean_tg_0 E1cyclostationary_mean_rr_4w_1,0.276,0.276,0.482,df_E1_noCMI,[0],[1],backward,717.581s
1,E1cyclostationary_mean_rr_12w_0 E1cyclostationary_mean_rr_1w_5 E1cyclostationary_mean_tg_6,0.254,0.254,0.471,df_E1_noCMI,[0],[1],forward,641.229s
2,E1cyclostationary_mean_rr_8w_4 E1cyclostationary_mean_tg_0,0.261,0.262,0.402,df_E1_noCMI,"[0, 1]",[1],backward,766.952s
3,E1cyclostationary_mean_rr_12w_0 E1cyclostationary_mean_tg_6 E1cyclostationary_mean_rr_8w_4,0.215,0.212,0.395,df_E1_noCMI,"[0, 1]",[1],forward,633.929s
4,E1cyclostationary_mean_rr_8w_4 E1cyclostationary_mean_tg_6,0.204,0.204,0.424,df_E1allfeatures_noCMI,[0],[1],backward,2451.030s
5,E1cyclostationary_mean_rr_12w_0 GMcyclostationary_mean_tg_0 E1cyclostationary_mean_rr_1w_5,0.281,0.281,0.484,df_E1allfeatures_noCMI,[0],[1],forward,2047.873s
6,E1cyclostationary_mean_rr_12w_4 E1cyclostationary_mean_rr_4w_1 E1cyclostationary_mean_tg_12w_0 E1cyclostationary_mean_tg_8w_4 E1cyclostationary_mean_tg_12w_7 E2cyclostationary_mean_tg_8w_2 E1cyclostationary_mean_tg_6,0.257,0.306,0.428,df_E1allfeatures_noCMI,"[0, 1]",[1],backward,2612.832s
7,E1cyclostationary_mean_rr_12w_0 E1cyclostationary_mean_tg_6 E1cyclostationary_mean_rr_8w_4,0.215,0.212,0.395,df_E1allfeatures_noCMI,"[0, 1]",[1],forward,2190.678s
8,E2cyclostationary_mean_tg_1 E2cyclostationary_mean_tg_3 E2cyclostationary_mean_tg_4 E2cyclostationary_mean_tg_8w_2,0.174,0.174,0.461,df_E2_noCMI,[0],[1],backward,148.270s
9,E2cyclostationary_mean_tg_1 E2cyclostationary_mean_rr_1w_3,0.198,0.198,0.440,df_E2_noCMI,[0],[1],forward,124.161s


### <a id='toc1_2_4_'></a>[TEFS as wrapper on E12GM](#toc0_)

In this way the method becomes a wrapper because we are making a selection solely by looking at the performance in regression.

In [26]:
# Load all TEFS simulations
results_files = sorted([file for file in os.listdir(constants.path_results) if file.endswith(".pkl")])
config_list = [file for file in results_files if file.split("_")[0] == "te"]
config_list

['te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
 'te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
 'te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
 'te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
 'te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
 'te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
 'te_e12gm_datasetdf_E1_noCMI_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
 'te_e12gm_datasetdf_E1_noCMI_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
 'te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
 'te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
 'te_e12gm_datasetdf_E1allf

Here I make two plots, one with a single line using fixed train and test, and one with cross-validation, in this case with `KFold`, but a version with `TimeSeriesSplit` is also available.

In [27]:
for config_name in config_list:
    basename = os.path.splitext(os.path.basename(config_name))[0]
    target_file_train_test = os.path.join(constants.path_figures, "tefs_as_wrapper", f"{basename}_wrapper.pdf")
    target_file_cv = os.path.join(constants.path_figures, "tefs_as_wrapper_cv", f"{basename}_wrapper_cv.pdf")
    if os.path.exists(target_file_train_test) and os.path.exists(target_file_cv):
        print(f"Skipping {config_name}...")
        continue

    print(f"Processing {config_name}...")

    # --------------------- Load simulation ---------------------
    simulation = file_management.load_from_pkl_file(
        os.path.join(
            constants.path_results,
            config_name,
        )
    )

    # --------------------- Load corresponding dataset ---------------------
    basin_name = config_name.split("_")[1]
    datasets, _ = datasets_and_configurations_loaders["te"].get(basin_name)()
    dataset_name = simulation["dataset_name"]
    dataframe = datasets[dataset_name]

    target_columns = ["target"]
    features_columns = dataframe["full"].drop(columns=target_columns).columns

    # --------------------- Select features using threshold (conservative) ---------------------
    selected_features_names_with_threshold = simulation["results"].select_features(simulation["params"]["threshold"])
    n_features_selected_with_threshold = len(selected_features_names_with_threshold)

    # --------------------- Compute test R2 for each number of features ---------------------
    test_r2_train_test = []
    test_r2_cv = []
    num_total_features = len(dataframe["full"].columns) - 1  # -1 because the last column is the target
    for num_features in range(0, num_total_features + 1):
        if num_features == 0:
            selected_features_names = []
        else:
            selected_features_names = simulation["results"].select_n_features(num_features)

        lagfeatures = simulation["params"]["lagfeatures"]
        lagtarget = simulation["params"]["lagtarget"]

        inputs_names_lags = {feature: lagfeatures for feature in selected_features_names}
        inputs_names_lags["target"] = lagtarget

        # --- Compute the train_test version ---
        test_r2_train_test.append(
            regression_analysis(
                inputs_names_lags=inputs_names_lags,
                target_name=target_columns[0],
                df_train=dataframe["train"],
                df_test=dataframe["test"],
            )
        )

        # --- Compute the cross-validation version ---
        # To perform a cross-validation, we need to concatenate the train and test sets
        unified_df = pd.concat([dataframe["train"], dataframe["test"]], axis=0).reset_index(drop=True)

        # Fixed window size
        # n_samples = unified_df.shape[0]
        # n_splits = 5
        # cv_scheme = TimeSeriesSplit(
        #     n_splits=n_splits,
        #     max_train_size=n_samples // (n_splits + 1),
        # )

        # Regular KFold
        cv_scheme = KFold(n_splits=4)  # 4 splits is about using the same test set size

        test_r2_cv.append(
            regression_analysis(
                inputs_names_lags=inputs_names_lags,
                target_name=target_columns[0],
                df=unified_df,
                cv_scheme=cv_scheme,
            )
        )

    test_r2_train_test = np.array(test_r2_train_test)
    test_r2_cv = np.array(test_r2_cv)

    # --------------------- Plot train test version ---------------------
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(test_r2_train_test, marker="o", label="Fixed train-test")
    maxima = np.where(test_r2_train_test == test_r2_train_test.max())[0]
    ax.plot(maxima, test_r2_train_test[maxima], marker="o", color="red", linestyle="None", label="Maximum", markersize=10)
    ax.plot(n_features_selected_with_threshold, test_r2_train_test[n_features_selected_with_threshold], marker="o", color="green", linestyle="None", label="TEFS (conservative)", markersize=10)
    ax.set_xlabel("Number of features")
    ax.set_ylabel("Test $R^2$")

    if simulation["params"]["threshold"] == np.inf:
        threshold_text = "\infty"
    elif simulation["params"]["threshold"] == -np.inf:
        threshold_text = "-\infty"
    else:
        threshold_text = simulation["params"]["threshold"]

    title_text = f"TEFS on basin {basin_name.upper()} with dataset {dataset_name}\n[lagfeatures $={simulation['params']['lagfeatures']}$, lagtarget $={simulation['params']['lagtarget']}$, direction = {simulation['params']['direction']}, threshold $={threshold_text}]$"
    ax.set_title(title_text)
    ax.legend()
    if num_total_features < 30:
        step = 1
    elif num_total_features < 80:
        step = 5
    else:
        step = 10
    ax.set_xticks(range(0, num_total_features + 1, step))
    ax.set_xticklabels(range(0, num_total_features + 1, step))
    ax.set_ylim(-0.1, 0.55)
    ax.grid()

    os.makedirs(os.path.dirname(target_file_train_test), exist_ok=True)
    plt.savefig(target_file_train_test, bbox_inches="tight")
    plt.close(fig)

    # --------------------- Plot cross-validation version ---------------------
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(test_r2_cv.mean(axis=1), marker="o", label="Cross-validation")
    maxima = np.where(test_r2_cv.mean(axis=1) == test_r2_cv.mean(axis=1).max())[0]
    ax.plot(maxima, test_r2_cv.mean(axis=1)[maxima], marker="o", color="red", linestyle="None", label="Maximum", markersize=10)
    ax.plot(n_features_selected_with_threshold, test_r2_cv.mean(axis=1)[n_features_selected_with_threshold], marker="o", color="green", linestyle="None", label="TEFS (conservative)", markersize=10)

    # plot confidence interval bands from cross-validation based on mean and standard deviation (90% confidence)
    alpha = 0.1
    quantile = scipy.stats.norm.ppf(1 - alpha / 2)
    ax.fill_between(range(test_r2_cv.shape[0]), test_r2_cv.mean(axis=1) - test_r2_cv.std(axis=1) * quantile / np.sqrt(test_r2_cv.shape[1]), test_r2_cv.mean(axis=1) + test_r2_cv.std(axis=1) * quantile / np.sqrt(test_r2_cv.shape[1]), alpha=0.3)

    ax.set_xlabel("Number of features")
    ax.set_ylabel("Test $R^2$")

    if simulation["params"]["threshold"] == np.inf:
        threshold_text = "\infty"
    elif simulation["params"]["threshold"] == -np.inf:
        threshold_text = "-\infty"
    else:
        threshold_text = simulation["params"]["threshold"]

    title_text = f"TEFS on basin {basin_name.upper()} with dataset {dataset_name}\n[lagfeatures $={simulation['params']['lagfeatures']}$, lagtarget $={simulation['params']['lagtarget']}$, direction = {simulation['params']['direction']}, threshold $={threshold_text}]$"
    ax.set_title(title_text)
    ax.legend()
    if num_total_features < 30:
        step = 1
    elif num_total_features < 80:
        step = 5
    else:
        step = 10
    ax.set_xticks(range(0, num_total_features + 1, step))
    ax.set_xticklabels(range(0, num_total_features + 1, step))
    ax.set_ylim(-0.1, 0.55)
    ax.grid()

    os.makedirs(os.path.dirname(target_file_cv), exist_ok=True)
    plt.savefig(target_file_cv, bbox_inches="tight")
    plt.close(fig)

Skipping te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1_noCMI_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_noCMI_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_l

### <a id='toc1_2_5_'></a>[Linking the wrapper to the original filter method](#toc0_)

Here I look for all configurations without CMI and match them to those with CMI. I show the plot above where I see the algorithm as a wrapper and highlight with vertical bars the points at which the variables chosen in the version with CMI were added/removed. There is also the option to choose variables manually (ideally the most common ones "by eye").

In [28]:
config_matches = []
for config_name in config_list:
    if "noCMI" in config_name and config_name.replace("_noCMI", "") in config_list:
        config_matches.append((config_name, config_name.replace("_noCMI", "")))

config_matches[:2]

[('te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl',
  'te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl'),
 ('te_e12gm_datasetdf_E1_noCMI_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl',
  'te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl')]

In [29]:
for config_name_noCMI, config_name in config_matches:
    basename = os.path.splitext(os.path.basename(config_name_noCMI))[0]
    target_file = os.path.join(constants.path_figures, "tefs_as_wrapper_mapping_filter", f"{basename}_wrapper_mapping_filter.pdf")
    if os.path.exists(target_file):
        print(f"Skipping {config_name}...")
        continue

    print(f"Processing {config_name}...")

    simulation_noCMI = file_management.load_from_pkl_file(
        os.path.join(
            constants.path_results,
            config_name_noCMI,
        )
    )

    simulation = file_management.load_from_pkl_file(
        os.path.join(
            constants.path_results,
            config_name,
        )
    )

    # feature selected with CMI
    basin_name = config_name.split("_")[1]
    datasets, _ = datasets_and_configurations_loaders["te"].get(basin_name)()
    dataset_name = simulation["dataset_name"]
    dataframe = datasets[dataset_name]
    target_columns = ["target"]
    features_columns = dataframe["full"].drop(columns=target_columns).columns

    selected_features_names_with_threshold = simulation["results"].select_features(simulation["params"]["threshold"])
    n_features_selected_with_threshold = len(selected_features_names_with_threshold)

    # choose manually
    selected_features_names_with_threshold = ["E1cyclostationary_mean_rr_4w_1", "E2cyclostationary_mean_tg_0"]

    # Load the noCMI version and process it
    basin_name = config_name_noCMI.split("_")[1]
    datasets, _ = datasets_and_configurations_loaders["te"].get(basin_name)()
    dataset_name_noCMI = simulation_noCMI["dataset_name"]
    dataframe_noCMI = datasets[dataset_name_noCMI]
    target_columns = ["target"]
    features_columns_noCMI = dataframe_noCMI["full"].drop(columns=target_columns).columns

    test_r2_train_test = []

    selected_features_names_previous = []  # new part
    corresponding_features_indexes = {}  # new part

    num_total_features = len(dataframe_noCMI["full"].columns) - 1  # -1 because the last column is the target
    for num_features in range(0, num_total_features + 1):
        if num_features == 0:
            selected_features_names = []
        else:
            selected_features_names_previous = selected_features_names.copy()  # new part
            selected_features_names = simulation_noCMI["results"].select_n_features(num_features)

            # if the feature that has been just added is in selected_features_names_with_threshold, add num_features to corresponding_features_indexes
            # looking at the set difference
            new_feature_name = list(set(selected_features_names).difference(set(selected_features_names_previous)))[0]
            if new_feature_name in selected_features_names_with_threshold:
                corresponding_features_indexes[num_features] = new_feature_name

        lagfeatures = simulation_noCMI["params"]["lagfeatures"]
        lagtarget = simulation_noCMI["params"]["lagtarget"]

        inputs_names_lags = {feature: lagfeatures for feature in selected_features_names}
        inputs_names_lags["target"] = lagtarget

        # --- Compute the train_test version ---
        test_r2_train_test.append(
            regression_analysis(
                inputs_names_lags=inputs_names_lags,
                target_name=target_columns[0],
                df_train=dataframe_noCMI["train"],
                df_test=dataframe_noCMI["test"],
            )
        )

    test_r2_train_test = np.array(test_r2_train_test)

    # --------------------- Plot ---------------------
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(test_r2_train_test, marker="o", label="Fixed train-test")

    # Get the default color cycle
    color_cycle = plt.rcParams["axes.prop_cycle"].by_key()["color"]

    # plot vertical lines in corresponding_features_indexes
    for i, (key, value) in enumerate(corresponding_features_indexes.items()):
        ax.axvline(x=key, linestyle="--", color=color_cycle[i + 1 % len(color_cycle)], label=f"{value}")

    maxima = np.where(test_r2_train_test == test_r2_train_test.max())[0]
    ax.plot(maxima, test_r2_train_test[maxima], marker="o", color="red", linestyle="None", label="Maximum", markersize=10)
    ax.plot(n_features_selected_with_threshold, test_r2_train_test[n_features_selected_with_threshold], marker="o", color="green", linestyle="None", label="TEFS (conservative)", markersize=10)
    ax.set_xlabel("Number of features")
    ax.set_ylabel("Test $R^2$")

    if simulation["params"]["threshold"] == np.inf:
        threshold_text = "\infty"
    elif simulation["params"]["threshold"] == -np.inf:
        threshold_text = "-\infty"
    else:
        threshold_text = simulation["params"]["threshold"]

    title_text = f"TEFS on basin {basin_name.upper()} with dataset {dataset_name}\n[lagfeatures $={simulation['params']['lagfeatures']}$, lagtarget $={simulation['params']['lagtarget']}$, direction = {simulation['params']['direction']}, threshold $={threshold_text}]$"
    ax.set_title(title_text)
    ax.legend()
    if num_total_features < 30:
        step = 1
    elif num_total_features < 80:
        step = 5
    else:
        step = 10

    ax.set_xticks(range(0, num_total_features + 1, step))
    ax.set_xticklabels(range(0, num_total_features + 1, step))
    ax.set_ylim(-0.1, 0.55)
    ax.grid()

    os.makedirs(os.path.dirname(target_file), exist_ok=True)
    plt.savefig(target_file, bbox_inches="tight")
    plt.close(fig)

Skipping te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0,1]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E1allfeatures_lagfeatures[0]_lagtarget[1]_directionforward_thresholdinf_k10.pkl...
Skipping te_e12gm_datasetdf_E2_lagfeatures[0,1]_lagtarget[1]_directionbackward_threshold0_k10.pkl...
Skipping te_e12gm_datasetdf_E2_lagfeatures[0,1]_lag